## Grupo 2 - Neo4J

Este trabalho consiste em implementar o esboço de um sistema de recomendação entre produtos de um e-commerce. Os blocos a seguir apresentam os códigos que devem ser inseridos no terminal do Neo4J para visualização dos resultados.

### Inserção de produtos

Utilizamos um arquivo .csv para inserir os dados da nossa base de dados, com mais de 10000 entradas.

In [ ]:
LOAD CSV WITH HEADERS FROM 'https://github.com/GuilhermeSGodoy/BDCD/blob/main/T2/Tabelas/myntra_products_catalog.csv' AS row
CREATE (p:Product {id: toInteger(row.ProductID), nome: row.ProductName, brand: row.ProducBrand, description: row.Description, price: row.Price, color: row.PrimaryColor, gender: row.Gender });

### Inserção de apenas 50 produtos (Recomendado)

Por fins de simplificação e economia de recursos, optamos por criar uma versão resumida da nossa base com 50 entradas, de modo que é a que utilizamos para manipular os testes que fizemos.

In [ ]:
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GuilhermeSGodoy/BDCD/main/T2/Tabelas/myntra_products_catalog-limit.csv' AS row
CREATE (p:Product {id: toInteger(row.ProductID), nome: row.ProductName, brand: row.ProducBrand, description: row.Description, price: row.Price, color: row.PrimaryColor, gender: row.Gender });

### Inserção de clientes

Criamos, também, uma lista com 100 clientes gerados aleatoriamente.

In [ ]:
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GuilhermeSGodoy/BDCD/main/T2/Tabelas/clientes.csv' AS row2
CREATE (c:Cliente {id: toInteger(row2.CPF), nome: row2.Nome, endereco: row2.Endereco, metodoPagamento: row2.MetodoPagamento, email: row2.Email, senha: row2.Senha, funcao: row2.Funcao });

### Criação do índice de proximidade entre produtos

A princípio, criamos um índice de proximidade entre os produtos da nossa base. A ideia é que cada produto tenha um valor (aqui gerado aleatoriamente, variando de 0 a 1) que o relacione com todos os outros produtos da base. NOTA: a visualização do Neo4J apresenta setas direcionadas entre um produto e outro, mas a relação gerada deve ser considerada como o índice entre os dois produtos indepedente de sua direção.

In [ ]:
MATCH (p1:Product), (p2:Product)
WHERE id(p1) < id(p2)
WITH p1, p2, rand() as indice
MERGE (p1)-[proximidade:PROXIMIDADE]->(p2)
ON CREATE SET proximidade.indice = indice
RETURN * LIMIT 10

### Um cliente aleatório compra três produtos aleatórios

Para exemplificar, criamos uma consulta que pega um cliente aleatório da nossa base e três produtos, também aleatórios, gerando uma relação de compra entre eles.

In [ ]:
MATCH (c:Cliente)
WHERE NOT (c)-[:COMPRA]->()
WITH c
LIMIT 1
MATCH (p:Product)
WITH c, p, rand() as r
ORDER BY r
LIMIT 3
MERGE (c)-[:COMPRA]->(p)
WITH c
MATCH (c)-[:COMPRA]->(p)
RETURN *

### Seleção de um produto aleatório e os cinco produtos com maior proximidade

Seguindo a lógica anterior, criamos uma consulta que seleciona um produto aleatório e apresenta os cinco primeiros produtos com maior proximidade com ele. NOTA: na visualização do Neo4J, o produto aleatório selecionado fica na parte inferior do grafo, e os outros produtos são dispostos usando o índice de maneira decrescente da direita para a esquerda.

In [ ]:
MATCH (p1:Product)
WITH p1, rand() as r
ORDER BY r
LIMIT 1
MATCH (p1)-[prox:PROXIMIDADE]->(p2:Product)
WITH p1, p2, prox.indice as indice
ORDER BY indice DESC
LIMIT 5
RETURN p1, COLLECT({product: p2, indice: indice}) as proximos_produtos